In [1]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split, GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = 15, 12

from sklearn.metrics import roc_curve, auc, roc_auc_score, precision_recall_curve, f1_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm

# import seaborn as sns

# from imblearn.over_sampling import SMOTE

In [79]:
df = pd.read_csv("/Users/ianchang/0.Workspaces/slaCoefficientProject/Origination Data/DTT SLA Summary - Overall Summary.csv").drop(columns=["Unnamed: 0", "partner_id", "partner", "xfr_request_id", "request_id", "true_SLA", "runtime_per_day", "payment_code"])

In [80]:
df['submit_date'] = pd.to_datetime(df['submit_date'])
df["submit_date"] = df["submit_date"].dt.floor('D')

# Extract year and month, then create the submit_month column
df['submit_month'] = df['submit_date'].dt.to_period('M')

In [81]:
origination_by_month = df[["on_time", "submit_month"]].groupby(by="submit_month").mean().reset_index()


In [82]:
# fig, ax = plt.subplots(figsize = (10,5))
# plt.style.use('bmh')

# ax.plot(origination_by_month["submit_month"].astype("str"), origination_by_month["on_time"], marker="v", label="Monthly OTD%", color="blue")


# ax.set_ylim(0.9, 1)
# ax.axhline(y=0.95, color='red', linestyle='--', label="Desired Level of OTD%")
# ax.set_title("Monthly Trend of On Time Delivery Percentage (OTD%)")
# ax.set_xlabel("Month")
# ax.set_ylabel("% in Decimals")

# plt.xticks(rotation=45)
# plt.legend()
# plt.show()

In [83]:
import random
import string

def get_random_string(length=8):
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

df['author'] = df['author'].str.replace(r'@.*', '', regex=True)

unique_authors = df['author'].unique()

author_mapping = {}

for author in unique_authors:
    author_mapping[author] = get_random_string()

df['author_anonymous'] = df['author'].map(author_mapping)



In [84]:
df = df.sort_values(by="author_anonymous")
df[df["asset_type"] == "LANGUAGE_TEMPLATE"]

,author,title_id,package_id,title_desc,asset_type,content_category,reason_code,original_audio_language,source_language,target_language,runtime_minutes,current_state,submit_date,due_date,start_date,on_time,billing_events,submit_month,author_anonymous
3959,alisonawoods,81517863,2064101,"Phantom Pups: Season 1: ""Phome Alone""",LANGUAGE_TEMPLATE,VIDEO,RUSH,en,--,en,18.8,AUTHORING_COMPLETE,2022-08-04,2022-08-04,2022-08-04,1,NaN,2022-08,AIYrnYvo
3957,alisonawoods,81517868,2064099,"Phantom Pups: Season 1: ""Fox Phoils Freddie""",LANGUAGE_TEMPLATE,VIDEO,RUSH,en,--,en,21.3,AUTHORING_COMPLETE,2022-08-02,2022-08-02,2022-07-27,1,NaN,2022-08,AIYrnYvo
3956,alisonawoods,81517867,2064097,"Phantom Pups: Season 1: ""Phreaky Phriday""",LANGUAGE_TEMPLATE,VIDEO,RUSH,en,--,en,22.8,AUTHORING_COMPLETE,2022-08-01,2022-08-01,2022-07-24,1,NaN,2022-08,AIYrnYvo
9448,alisonawoods,81474598,2033264,The Joys and Sorrows of Young Yuguo,LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,28.3,AUTHORING_COMPLETE,2022-07-18,2022-07-19,2022-07-12,1,NaN,2022-07,AIYrnYvo
10250,alisonawoods,81455917,2067669,"Rise of Empires: Ottoman: Season 2: ""Night Att...",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,43.2,AUTHORING_COMPLETE,2022-08-31,2022-08-31,2022-08-12,1,NaN,2022-08,AIYrnYvo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6944,coenrees,81487585,1997309,"Riverdale: Season 6: ""Chapter One Hundred and ...",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,42.2,AUTHORING_COMPLETE,2022-05-05,2022-05-05,2022-05-03,1,NaN,2022-05,yWKrgnFc
6865,coenrees,81034895,2037268,Players,LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,103.3,AUTHORING_COMPLETE,2022-05-04,2022-05-04,2022-04-23,1,NaN,2022-05,yWKrgnFc
11081,coenrees,81466101,2097779,"Wellmania: Season 1: ""Episode 4""",LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,34.8,AUTHORING_COMPLETE,2022-10-25,2022-10-25,2022-10-24,1,NaN,2022-10,yWKrgnFc
6724,coenrees,81311995,2055291,Joel Kim Booster: Psychosexual,LANGUAGE_TEMPLATE,VIDEO,STANDARD,en,--,en,67.4,AUTHORING_COMPLETE,2022-04-28,2022-04-28,2022-04-21,1,NaN,2022-04,yWKrgnFc


In [85]:
# df.groupby(by=["author_anonymous", "asset_type", "reason_code", "source_language", "target_language"]).agg({"runtime_minutes": "sum"}).head(50)

In [86]:
# Initialize  
df["expected_daily_output"] = 0

for index, row in df.iterrows():
    if row["asset_type"] == "SUBTITLE":
        if row["reason_code"] == "STANDARD":
            df.at[index, "expected_daily_output"] = 15
        elif row["reason_code"] == "RUSH":
            df.at[index, "expected_daily_output"] = 30

    elif row["asset_type"] == "CLOSED_CAPTIONING":
        if row["original_audio_language"] == row["target_language"]:
            if row["reason_code"] == "STANDARD":
                df.at[index, "expected_daily_output"] = 30
            elif row["reason_code"] == "RUSH":
                df.at[index, "expected_daily_output"] = 60
        else:
            if row["reason_code"] == "STANDARD":
                df.at[index, "expected_daily_output"] = 15
            elif row["reason_code"] == "RUSH":
                df.at[index, "expected_daily_output"] = 30
    
    elif row["asset_type"] == "LANGUAGE_TEMPLATE":
        if row["reason_code"] == "STANDARD":
            df.at[index, "expected_daily_output"] = 15
        elif row["reason_code"] == "RUSH":
            df.at[index, "expected_daily_output"] = 30

df = df[['author_anonymous','title_id', 'package_id', 'title_desc', 'asset_type',
       'content_category', 'reason_code', 'original_audio_language',
       'source_language', 'target_language',"expected_daily_output", 'runtime_minutes',
        'start_date', 'submit_date', 'due_date', 'on_time',
       'billing_events']]

In [87]:
import pandas as pd

def calculate_business_days(start_date_str, end_date_str):
    """
    Calculate the number of business days between two dates excluding weekends.

    Parameters:
    start_date_str (str): The start date in YYYY-MM-DD format.
    end_date_str (str): The end date in YYYY-MM-DD format.

    Returns:
    int: The number of business days between the two dates.
    """
    # Convert the date strings to datetime objects
    start_date = pd.to_datetime(start_date_str)
    end_date = pd.to_datetime(end_date_str)
    
    # Calculate the number of business days (excluding weekends)
    business_days = pd.date_range(start=start_date, end=end_date, freq='B')
    
    # Return the count of business days
    return len(business_days)


In [90]:
for index, row in df.iterrows():
    df.at[index,"num_days_spent"] = calculate_business_days(row["start_date"], row["submit_date"])

df = df[['author_anonymous','title_id', 'package_id', 'title_desc', 'asset_type',
       'content_category', 'reason_code', 'original_audio_language',
       'source_language', 'target_language',"expected_daily_output", 'runtime_minutes',
        'start_date', 'submit_date','num_days_spent', 'due_date', 'on_time',
       'billing_events']]

In [91]:
df.head()

,author_anonymous,title_id,package_id,title_desc,asset_type,content_category,reason_code,original_audio_language,source_language,target_language,expected_daily_output,runtime_minutes,start_date,submit_date,num_days_spent,due_date,on_time,billing_events
4615,AADQWVjI,81387029,2014903,"The Hidden Lives of Pets: Season 1: ""Episode 3""",SUBTITLE,VIDEO,STANDARD,en,en,zh-Hant,15,31.3,2022-01-14,2022-01-25,8.0,2022-01-25,1,511.0
8968,AADQWVjI,81317476,2000070,"Deadwind: Season 3: ""Episode 5""",SUBTITLE,VIDEO,STANDARD,fi,en,zh-Hant,15,44.6,2022-06-28,2022-06-30,3.0,2022-06-30,1,464.0
9699,AADQWVjI,81317478,2000077,"Deadwind: Season 3: ""Episode 7""",SUBTITLE,VIDEO,STANDARD,fi,en,zh-Hant,15,43.9,2022-07-13,2022-07-25,9.0,2022-07-25,1,438.0
6161,AADQWVjI,81036178,2019499,"Rebelde: Season 2: ""Episode 4""",SUBTITLE,VIDEO,STANDARD,es,en,zh-Hant,15,37.2,2022-03-31,2022-04-07,6.0,2022-04-07,1,519.0
5327,AADQWVjI,81203515,2022449,"Wild Babies: Season 1: ""New Arrivals""",SUBTITLE,VIDEO,STANDARD,en,en,zh-Hant,15,33.2,2022-03-03,2022-03-07,3.0,2022-03-07,1,244.0
